In [5]:
import numpy as np

In [6]:
class HiddenMarkovModel:
    def __init__(self, num_states, num_obs):
        self.num_states = num_states
        self.num_obs = num_obs
        self.transition_prob = np.ones((num_states, num_states)) / num_states
        self.emission_prob = np.ones((num_states, num_obs)) / num_obs
        self.initial_prob = np.ones(num_states) / num_states
    
    def train(self, observations, states, num_iterations=100):
        for _ in range(num_iterations):
            new_transition_prob = np.zeros((self.num_states, self.num_states))
            new_emission_prob = np.zeros((self.num_states, self.num_obs))
            new_initial_prob = np.zeros(self.num_states)
            
            for observation, state in zip(observations, states):
                alpha, beta = self.forward_backward(observation)
                new_initial_prob[state[0]] += 1
                for t in range(len(observation)-1):
                    new_transition_prob[state[t], state[t+1]] += alpha[t, state[t]] * beta[t+1, state[t+1]]
                    new_emission_prob[state[t], observation[t]] += 1

            self.transition_prob = new_transition_prob / new_transition_prob.sum(axis=1)[:, np.newaxis]
            self.emission_prob = new_emission_prob / new_emission_prob.sum(axis=1)[:, np.newaxis]
            self.initial_prob = new_initial_prob / new_initial_prob.sum()

    def forward_backward(self, observation):
        T = len(observation)
        alpha = np.zeros((T, self.num_states))
        beta = np.zeros((T, self.num_states))

        # Forward pass
        alpha[0] = self.initial_prob * self.emission_prob[:, observation[0]]
        for t in range(1, T):
            alpha[t] = np.dot(alpha[t-1], self.transition_prob) * self.emission_prob[:, observation[t]]

        # Backward pass
        beta[-1] = 1
        for t in range(T-2, -1, -1):
            beta[t] = np.dot(self.transition_prob, self.emission_prob[:, observation[t+1]] * beta[t+1])

        return alpha, beta
    
    def predict(self, observation):
        alpha, beta = self.forward_backward(observation)
        return np.argmax(alpha * beta, axis=1)

In [7]:
num_states = 2
num_obs = 3
hmm = HiddenMarkovModel(num_states, num_obs)

observations = [[0, 1, 2, 1, 0], [2, 1, 0, 0, 2]]
states = [[0, 0, 1, 1, 1], [1, 1, 0, 0, 1]]

hmm.train(observations, states)

In [8]:
new_observation = [1, 2, 0, 1]
predicted_states = hmm.predict(new_observation)
predicted_states

array([1, 1, 0, 1])